<h1 align = "center"><b> Análise de Transformações </b></h1>

**Objetivo Geral:** Construir modelos capazes de prever a demanda do produto 1 da loja 1, e assim, auxiliar uma empresa a otimizar suas decisões logísticas.

**Objetivo Específico:** Essa é a primeira etapa desse estudo de caso. Seu objetivo é analisar os dados e identificar qual/quais transformações devem ser aplicadas para obter uma série estacionária e com distribuição aproximadamente normal.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import statsmodels.tsa.stattools as stattools 
import sklearn.preprocessing as skpp

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [5]:
df = pd.read_csv('../Demanda Diária.csv', index_col = 0, parse_dates = True)
df = df.loc[df['loja'] == 1]
df = df.loc[df['produto'] == 1]
df = df.drop(['loja', 'produto'], axis = 1)

série = pd.Series(df['itens_vendidos'], index = df.index)

In [6]:
série.head()

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64

In [9]:
série.describe()

count    1826.000000
mean       19.971522
std         6.741022
min         4.000000
25%        15.000000
50%        19.000000
75%        24.000000
max        50.000000
Name: itens_vendidos, dtype: float64

<h2 align="center"><b> Funções para Cálculo das Métricas </b></h2>

### Estacionariedade:

In [ ]:
def visualizar_estacionaridade (série, nome_série):
	plt.plot(série, label = 'Série Real')
	plt.plot(série.rolling(12).mean(), label = 'Média Móvel')
	plt.title(f'Visualizar Estacionaridade - {nome_série}')
	plt.legend(loc = 'best')
	plt.show()

visualizar_estacionaridade(série, 'Série Original')

![Visualizar Estacionariedade](./Gráficos/Série%20Original/Estacionaridade.png)
&emsp;&emsp; Visualmente, a média parece variar com o tempo. Provavelmente essa série não é estacionária. Para confirmar, vamos utilizar o teste KPSS e o teste de Dickey-Fuller Aumentado (ADF).

In [105]:
def teste_KPSS(série):
    kpss = stattools.kpss(série)
    print(f'Estatística de Teste = {kpss[0]}')
    print(f'p-valor = {kpss[1]}')
    print(f'Valores Críticos:')
    for chave, valor in kpss[3].items():
        print(f'    {chave}: {valor}')
    print(f'Resultado: {"Temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente é estacionária" if (kpss[0] < kpss[3]["5%"]) else "Não temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente não é estacionária"}.')

teste_KPSS(série)

Estatística de Teste = 1.891425278325667
p-valor = 0.01
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Não temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente não é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss = stattools.kpss(série)


In [106]:
def teste_adfuller(serie_temporal):
    adf = stattools.adfuller(serie_temporal)
    print(f"Estatística ADF: {adf[0]}")
    print(f"Valor p: {adf[1]}")
    print("Valores críticos:")
    for chave, valor in adf[4].items():
        print(f"   {chave}: {valor}")
    print(f'Resultado: {"Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária" if (adf[0] < adf[4]["5%"]) else "Não temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente não é estacionária"}.')

teste_adfuller(série)

Estatística ADF: -3.1576705563328193
Valor p: 0.022569380626570587
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; A visualização por gráfico e o teste KPSS concordam que a série não é estacionária. O teste ADF discorda. Então, vamos considerar que a série não é estacionária.

### Normalidade:

In [ ]:
def visualizar_normalidade(série, nome_série = 'Série'):
    stats.probplot(série, dist = 'norm', plot = plt)
    plt.title(f'Normal QQ Plot - {nome_série}') 
    plt.show()

    sns.histplot(série, kde = True)
    plt.title(f'Histograma - {nome_série}')
    plt.show()

visualizar_normalidade(série, 'Série Original')

![Normal QQ Plot](./Gráficos/Série%20Original/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20Original/Histograma.png)
&emsp;&emsp; Visualmente, a série não parece ter uma distribuição normal. Para confirmar, vamos utilizar o teste de Shapiro-Wilk.

In [108]:
def teste_shapiro(série):
    e, p = stats.shapiro(série)
    print(f'Estatística de Teste = {e}')
    print(f'p-valor = {p}')
    print(f'Resultado: {"Não temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente possui distribuição Normal" if (p > 0.05) else "Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal"}')

teste_shapiro(série)


Estatística de Teste = 0.9879128336906433
p-valor = 3.0621737784342073e-11
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


&emsp;&emsp; Tanto a análise por gráfico quanto o teste de Shapiro-Wilk concordam que a série não possui distribuição normal.

<h2 align="center"><b> Transformação por Diferenciação </b></h2>
&emsp;&emsp; A série original não é estacionária. Uma transformação comum para resolver esse problema é a diferenciação. Quando aplicamos a diferenciação, a série resultante passa a medir a variação entre os valores da série original. Caso a série resultante não seja estacionária, podemos aplicar a diferenciação novamente.

In [109]:
série_diff = série.diff().dropna()

In [110]:
série_diff.head()

data
2013-01-02   -2.0
2013-01-03    3.0
2013-01-04   -1.0
2013-01-05   -3.0
2013-01-06    2.0
Name: itens_vendidos, dtype: float64

In [111]:
série_diff.describe()

count    1825.000000
mean        0.005479
std         7.085550
min       -26.000000
25%        -4.000000
50%         0.000000
75%         5.000000
max        25.000000
Name: itens_vendidos, dtype: float64

### Estacionariedade:

In [ ]:
visualizar_estacionaridade(série_diff, 'Série Diferenciada')

![Visualizar Estacionariedade](./Gráficos/Série%20Diferenciada/Estacionaridade.png)
&emsp;&emsp; Visualmente, a média parece constante. Isso é um bom sinal, provavelmente a série é estacionária.

In [115]:
teste_KPSS(série_diff)

Estatística de Teste = 0.03333614198230825
p-valor = 0.1
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss = stattools.kpss(série)


In [16]:
teste_adfuller(série_diff)

Estatística ADF: -12.676793886047477
Valor p: 1.2109276320436821e-23
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; Claramente, a transformação por diferenciação melhorou a estacionariedade da série. Os três testes concordam que a série é estacionária após a transformação.

### Normalidade:

In [ ]:
visualizar_normalidade(série_diff, 'Série Diferenciada')

![Normal QQ Plot](./Gráficos/Série%20Diferenciada/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20Diferenciada/Histograma.png)

&emsp;&emsp; Visualmente, a série parece ter se aproximado mais de uma distribuição normal em relação à série original. A diferenciação não tem como objetivo normalizar a série, para esse fim existem outras transformações mais adequadas.

In [117]:
teste_shapiro(série_diff)

Estatística de Teste = 0.9936521649360657
p-valor = 4.634608785636374e-07
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


### Desfazendo a Transformação:
&emsp;&emsp; Para revertermos a transformação, devemos somar todas as variações anteriores e adicionar o valor inicial da série original. Podemos fazer isso manualmente ou utilizando a função `cumsum()` do pandas.

In [121]:
série_invertida = série_diff.cumsum() + série.iloc[0]
display(série_invertida.head())

data
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
2013-01-06    12.0
Name: itens_vendidos, dtype: float64

In [118]:
def Inverter_Diferenciação(série, valor_inicial):
    série_invertida = [valor_inicial]
    for i in range(0, len(série)):
        série_invertida.append(série_invertida[i] + série[i])
    série_invertida = pd.Series(série_invertida[1:], index = série.index).dropna()
    return série_invertida

série_invertida = Inverter_Diferenciação(série_diff, série.iloc[0])

In [119]:
display(série_invertida.head()) # A informação do primeiro valor foi perdida, mas o restante da informação se manteve intacta.
display(série.head())

data
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
2013-01-06    12.0
dtype: float64

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64

<h2 align="center"><b> Transformação por Logaritmo </b></h2>
&emsp;&emsp; A transformação por logaritmo é uma transformação comum para tornar a série mais próxima de uma distribuição normal. Essa transformação é útil quando a série possui uma tendência exponencial.

In [21]:
série_log = np.log(série)

In [122]:
série_log.head()

data
2013-01-01    2.564949
2013-01-02    2.397895
2013-01-03    2.639057
2013-01-04    2.564949
2013-01-05    2.302585
Name: itens_vendidos, dtype: float64

In [123]:
série_log.describe()

count    1826.000000
mean        2.932441
std         0.365107
min         1.386294
25%         2.708050
50%         2.944439
75%         3.178054
max         3.912023
Name: itens_vendidos, dtype: float64

### Estacionariedade:

In [ ]:
visualizar_estacionaridade(série_log, 'Série Logarítmica')

![Visualizar Estacionariedade](./Gráficos/Série%20Logarítmica/Estacionaridade.png)

In [125]:
teste_KPSS(série_log)

Estatística de Teste = 1.819823815566604
p-valor = 0.01
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Não temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente não é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss = stattools.kpss(série)


In [126]:
teste_adfuller(série_log)

Estatística ADF: -3.5944235528561648
Valor p: 0.005868536747948788
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; Os resultados da transformação por logaritmo foram bastante similares aos da série original. Isso é esperado, pois essa transformação é feita para melhorar a normalidade da série, e não a estacionariedade.

### Normalidade:

In [ ]:
visualizar_normalidade(série_log, 'Série Logarítmica')

![Normal QQ Plot](./Gráficos/Série%20Logarítmica/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20Logarítmica/Histograma.png)
&emsp;&emsp; Visualmente, a série parece ter se aproximado mais de uma distribuição normal em relação à série original, mas provavelmente não o suficiente. Para confirmar, vamos utilizar o teste de Shapiro-Wilk.

In [28]:
teste_shapiro(série_log)

Estatística de Teste = 0.9755350947380066
p-valor = 4.101878477192872e-17
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


&emsp;&emsp; A transformação por logaritmo melhorou a normalidade da série, mas não o suficiente para que ela seja considerada normal. Talvez outras transformações que tentem melhorar a normalidade da série possam ser mais efetivas (como a transformação Box-Cox ou a transformação por raiz cúbica).

### Desfazendo a Transformação:
&emsp;&emsp; Para invertermos a transformação, basta elevarmos a base do logaritmo a cada valor da série. A função log do numpy utiliza a base natural (e), então podemos utilizar a função `exp()` do numpy para inverter a transformação.

In [128]:
série_invertida = np.exp(série_log)

In [129]:
display(série_invertida.head())
display(série.head())

data
2013-01-01    13.0
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
Name: itens_vendidos, dtype: float64

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64

<h2 align="center"><b> Transformação por Raiz Cúbica </b></h2>
&emsp;&emsp; A transformação por raiz cúbica, como o próprio nome diz, consiste em aplicar a raiz cúbica a cada valor da série. A transformação pode ser utilizada com outros coeficientes (como raiz quadrada, raiz quarta, etc), mas é importante que o coeficiente seja ímpar caso a série possua valores negativos. Essa transformação é útil para tornar a série mais próxima de uma distribuição normal.

In [130]:
série_cub = série ** (1/3) # Elevar a série a 1/3 é equivalente a fazer a raiz cúbica da série.

In [131]:
série_cub.head()

data
2013-01-01    2.351335
2013-01-02    2.223980
2013-01-03    2.410142
2013-01-04    2.351335
2013-01-05    2.154435
Name: itens_vendidos, dtype: float64

In [132]:
série_cub.describe()

count    1826.000000
mean        2.676997
std         0.314813
min         1.587401
25%         2.466212
50%         2.668402
75%         2.884499
max         3.684031
Name: itens_vendidos, dtype: float64

### Estacionariedade:

In [ ]:
visualizar_estacionaridade(série_cub, 'Série Cúbica')

![Visualizar Estacionariedade](./Gráficos/Série%20Cúbica/Estacionaridade.png)
&emsp;&emsp; Visualmente, não há nenhuma mudança em relação à série original. Isso é esperado, pois essa transformação é feita para melhorar a normalidade da série, e não a estacionariedade. 

In [136]:
teste_KPSS(série_cub)

Estatística de Teste = 1.851053858920974
p-valor = 0.01
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Não temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente não é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss = stattools.kpss(série)


In [137]:
teste_adfuller(série_cub)

Estatística ADF: -3.421812989651452
Valor p: 0.010237366962745204
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; Assim como a transformação por logaritmo, a transformação por raiz cúbica não melhorou a estacionariedade da série. Isso é esperado, pois essa transformação é feita para melhorar a normalidade da série, e não a estacionariedade.

### Normalidade:

In [ ]:
visualizar_normalidade(série_cub, 'Série Cúbica')

![Normal QQ Plot](./Gráficos/Série%20Cúbica/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20Cúbica/Histograma.png)
&emsp;&emsp; Visualmente, a série parece ter se aproximado mais ainda de uma distribuição normal em relação à série logarítmica, mas provavelmente não o suficiente. Para confirmar, vamos utilizar o teste de Shapiro-Wilk.

In [139]:
teste_shapiro(série_cub)

Estatística de Teste = 0.9935336112976074
p-valor = 3.6200029285282653e-07
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


&emsp;&emsp; A transformação por raiz cúbica melhorou a normalidade da série, mas não o suficiente para que ela seja considerada normal. O resultado foi melhor que o da transformação por logaritmo, mas ainda não é o ideal.

### Desfazendo a Transformação:
&emsp;&emsp; Para invertermos a transformação, basta elevarmos a cada valor da série ao cubo.

In [140]:
série_invertida = série_cub ** 3

In [141]:
display(série_invertida.head())
display(série.head())

data
2013-01-01    13.0
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
Name: itens_vendidos, dtype: float64

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64

<h2 align="center"><b> Transformação por Box-Cox </b></h2>
&emsp;&emsp; A transformação Box-Cox é uma transformação que tenta melhorar a normalidade da série por meio de uma função com um parâmetro ajustável. Caso não seja definido um parâmetro, a função tenta encontrar o melhor parâmetro para maximizar a normalidade da série. 

A função Box-Cox é definida como: $y(\lambda) = \begin{cases} \frac{{y^\lambda - 1}}{{\lambda}}, & \text{se }\lambda \neq 0 \\ \log(y), & \text{se }\lambda = 0 \end{cases}$


In [142]:
série_boxcox, lambda_encontrado = stats.boxcox(série)
série_boxcox = pd.Series(série_boxcox, index = série.index)
print(f'Lambda encontrado: {lambda_encontrado}')

Lambda encontrado: 0.576185039143375


In [143]:
série_boxcox.head()

data
2013-01-01    5.872534
2013-01-02    5.174365
2013-01-03    6.204434
2013-01-04    5.872534
2013-01-05    4.805129
dtype: float64

In [144]:
série_boxcox.describe()

count    1826.000000
mean        7.867947
std         1.914343
min         2.122213
25%         6.526428
50%         7.731977
75%         9.096108
max        14.797730
dtype: float64

### Estacionariedade:

In [ ]:
visualizar_estacionaridade(série_boxcox, 'Série Box-Cox')

![Visualizar Estacionariedade](./Gráficos/Série%20Box-Cox/Estacionaridade.png)

In [146]:
teste_KPSS(série_boxcox)

Estatística de Teste = 1.8690289994355702
p-valor = 0.01
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Não temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente não é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss = stattools.kpss(série)


In [147]:
teste_adfuller(série_boxcox)

Estatística ADF: -3.3129671697156997
Valor p: 0.014308698631174052
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; Como esperado, essa transformação também não melhorou a estacionariedade da série. Como as duas transformações anteriores, essa transformação é feita para melhorar a normalidade da série, e não a estacionariedade.

### Normalidade:

In [ ]:
visualizar_normalidade(série_boxcox, 'Série Box-Cox')

![Normal QQ Plot](./Gráficos/Série%20Box-Cox/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20Box-Cox/Histograma.png)

&emsp;&emsp; O resultado da transformação Box-Cox foi o melhor até agora. A série parece ter se aproximado mais de uma distribuição normal em relação às outras transformações. Para confirmar, vamos utilizar o teste de Shapiro-Wilk.

In [149]:
teste_shapiro(série_boxcox)

Estatística de Teste = 0.997368574142456
p-valor = 0.0037319986149668694
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


&emsp;&emsp; Infelizmente, a transformação por Box-Cox também não resolveu completamente o problema da normalidade da série. Mesmo assim o resultado se aproximou bastante do ideal.

### Desfazendo a Transformação:
&emsp;&emsp; Para invertermos a transformação, basta fazermos a inversa da função Box-Cox.

In [152]:
def Inverter_BoxCox(série, lambda_encontrado):
    if lambda_encontrado == 0: return np.exp(série)
    else: return (série * lambda_encontrado + 1) ** (1 / lambda_encontrado)

série_invertida = Inverter_BoxCox(série_boxcox, lambda_encontrado)

In [153]:
display(série_invertida.head())
display(série.head())

data
2013-01-01    13.0
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
dtype: float64

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64

<h2 align="center"><b> MinMaxScaler </b></h2>

&emsp;&emsp; Por fim, mas não menos importante, temos o MinMaxScaler. Diferentemente de todas as transformações anteriores, essa transformação não melhora a estacionariedade ou a normalidade da série.

&emsp;&emsp; Essa transformação não beneficia os modelos que vamos utilizar nesse projeto, mas é uma transformação muito útil para modelos de aprendizado de máquina. O objetivo dessa transformação é colocar todos os valores numéricos da série entre 0 e 1. Isso é especialmente útil quando os dados possuem escalas muito diferentes, pois isso pode prejudicar o aprendizado de máquina. 

A fórmula utilizada é: $X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$

&emsp;&emsp; Uma observação importante é a utilização do método `reshape(-1, 1)` para que a função funcione corretamente no nosso caso. Isso é necessário pois a função espera um array bidimensional, mas como estamos trabalhando com uma série temporal univariada, o array é unidimensional.

In [154]:
padronizador = skpp.MinMaxScaler()
série_minmax = padronizador.fit_transform(série.values.reshape(-1, 1))
série_minmax = pd.Series(série_minmax[:, 0], index = série.index)

In [155]:
série_minmax.head()

data
2013-01-01    0.195652
2013-01-02    0.152174
2013-01-03    0.217391
2013-01-04    0.195652
2013-01-05    0.130435
dtype: float64

In [156]:
série_minmax.describe()

count    1826.000000
mean        0.347207
std         0.146544
min         0.000000
25%         0.239130
50%         0.326087
75%         0.434783
max         1.000000
dtype: float64

### Estacionariedade:

In [ ]:
visualizar_estacionaridade(série_minmax, 'Série MinMaxScaler')

![Visualizar Estacionariedade](./Gráficos/Série%20MinMaxScaler/Estacionaridade.png)

In [55]:
teste_KPSS(série_minmax)

Estatística de Teste = 1.8914252783255971
p-valor = 0.01
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Não temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente não é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss = stattools.kpss(série)


In [56]:
teste_adfuller(série_minmax)

Estatística ADF: -3.157670556332835
Valor p: 0.022569380626569536
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; Como esperado, essa transformação basicamente não alterou a estacionariedade da série.

### Normalidade:

In [ ]:
visualizar_normalidade(série_minmax, 'Série MinMaxScaler')

![Normal QQ Plot](./Gráficos/Série%20MinMaxScaler/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20MinMaxScaler/Histograma.png)

In [160]:
teste_shapiro(série_minmax)

Estatística de Teste = 0.9879128336906433
p-valor = 3.0621737784342073e-11
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


&emsp;&emsp; Assim como a estacionariedade, essa transformação basicamente não alterou a normalidade da série. A unica diferença é que a escala do eixo x do gráfico mudou.

### Desfazendo a Transformação:
&emsp;&emsp; Para invertermos a transformação, basta utilizarmos a função `inverse_transform()` do sklearn.preprocessing no padronizador que utilizamos para fazer a transformação.

In [161]:
série_invertida = padronizador.inverse_transform(série_minmax.values.reshape(-1, 1))
série_invertida = pd.Series(série_invertida[:, 0], index = série.index)

In [162]:
display(série_invertida.head())
display(série.head())

data
2013-01-01    13.0
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
dtype: float64

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64

<h2 align="center"><b> Conclusão do Pré-Processamento </b></h2>
&emsp;&emsp; A série original possui problemas na estacionariedade e na normalidade. Como cada transformação resolve um desses problemas, vamos utilizar duas transformações. A diferenciação foi a única que resultou em uma melhora na estacionariedade da série, enquanto a transformação utilizando o método Box-Cox demonstrou ser a mais eficaz para se aproximar de uma distribuição normal. Quanto ao MinMaxScaler, não será aplicado neste projeto, uma vez que nenhum dos nossos modelos se beneficiaria dessa transformação.

In [163]:
def Transformar_Série(série):
    série_transformada, lambda_encontrado = stats.boxcox(série)
    série_transformada = pd.Series(série_transformada, index = série.index)
    print(f'Lambda encontrado: {lambda_encontrado}')

    série_transformada = série_transformada.diff().dropna()
    return série_transformada, lambda_encontrado

série_transformada, lambda_encontrado = Transformar_Série(série)

Lambda encontrado: 0.576185039143375


In [164]:
série_transformada.head()

data
2013-01-02   -0.698170
2013-01-03    1.030070
2013-01-04   -0.331900
2013-01-05   -1.067405
2013-01-06    0.724493
dtype: float64

In [165]:
série_transformada.describe()

count    1825.000000
mean        0.001623
std         2.005487
min        -7.231687
25%        -1.290260
50%         0.000000
75%         1.337940
max         6.171877
dtype: float64

### Estacionariedade:

In [ ]:
visualizar_estacionaridade(série_transformada, 'Série Transformada Final')

![Visualizar Estacionariedade](./Gráficos/Série%20Diferenciada/Estacionaridade.png)

In [167]:
teste_KPSS(série_transformada)

Estatística de Teste = 0.026638890198529906
p-valor = 0.1
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste KPSS, a série provavelmente é estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_7984\1383781043.py:2: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss = stattools.kpss(série)


In [78]:
teste_adfuller(série_transformada)

Estatística ADF: -12.60636138570757
Valor p: 1.692462406965177e-23
Valores críticos:
   1%: -3.4339840952648695
   5%: -2.8631452508003057
   10%: -2.567624583142913
Resultado: Temos evidencias contra a hipótese nula. Então segundo o teste de Dickey-Fuller Aumentado, a série provavelmente é estacionária


&emsp;&emsp; Os testes de estacionariedade concordam que a série é estacionária após as duas transformações.

### Normalidade:

In [ ]:
visualizar_normalidade(série_transformada, 'Série Transformada Final')

![Normal QQ Plot](./Gráficos/Série%20Transformada%20Final/Normal%20QQ%20Plot.png)
![Histograma](./Gráficos/Série%20Transformada%20Final/Histograma.png)

In [169]:
teste_shapiro(série_transformada)

Estatística de Teste = 0.9954820871353149
p-valor = 2.6225739929941483e-05
Resultado: Temos evidências para rejeitar a hipótese nula. Então segundo o teste de Shapiro-Wilk, a série provavelmente não possui distribuição Normal


&emsp;&emsp; Infelizmente, a combinação não conseguiu resolver completamente os problemas da série. Ainda assim, a série transformada está significativamente melhor que a original, e isso deve ajudar bastante no desempenho dos modelos.

### Desfazendo as Transformações:
&emsp;&emsp; Para invertermos as transformações, basta aplicarmos as inversas na ordem inversa. Como aplicamos a transformação Box-Cox e depois a diferenciação, devemos aplicar a inversa da diferenciação e depois a inversa da transformação Box-Cox.

In [170]:
def Inverter_Transformação(série_transformada, lambda_encontrado, valor_inicial):
    série_invertida = [stats.boxcox(valor_inicial, lambda_encontrado)]
    for i in range(0, len(série_transformada)):
        série_invertida.append(série_invertida[i] + série_transformada[i])
    série_invertida = pd.Series(série_invertida[1:], index = série_transformada.index).dropna()

    if lambda_encontrado == 0: return np.exp(série_invertida)
    else: return (série_invertida * lambda_encontrado + 1) ** (1 / lambda_encontrado)

série_invertida = Inverter_Transformação(série_transformada, lambda_encontrado, série.iloc[0])

In [171]:
display(série_invertida.head()) # A informação do primeiro elemento é perdida na Diferenciação
display(série.head())

data
2013-01-02    11.0
2013-01-03    14.0
2013-01-04    13.0
2013-01-05    10.0
2013-01-06    12.0
dtype: float64

data
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: itens_vendidos, dtype: int64